# Simple Matching Engine and Order Book

The code below aims to replicate a simple matching engine, which matches bid and ask orders within respective bid and ask books. Some of this code, notably the classes defining incoming orders and some of the TestOrderBook class have been taken from class notes from FINM25000 at Uchicago.

In [1]:
import time 
from abc import ABC
from enum import Enum
import numpy as np

We first create simple classes that define the possible order type and order side (buy or sell), as well as the possible exceptions that can occur with regards to the input order. The possible order types considered are:
1. $\textbf{Limit orders}$: it is a direction to buy/sell a security at a specified price or better. A buy (sell) limit order will be executed only at the limit price or a lower (higher) price. In this type, the price is guaranteed, but the filling of the order is not. 
2. $\textbf{Market orders}$: calls for a trade at the best available price in the current market. the filling is guaranteed, but the price is not. 
3. $\textbf{IOC orders}$: Immediate or Cancel (IOC) orders cancel any part of the order that doesn't fill immediately. It is typically used when submitting a large order to avoid having it filled at an array of prices.
Other orders not considered are FOK and AON orders. The classes below simply enumerate the order types and sides for easier order labeling.

In [2]:
class OrderType(Enum):
    LIMIT = 1
    MARKET = 2
    IOC = 3

class OrderSide(Enum):
    BUY = 1
    SELL = 2

class NonPositiveQuantity(Exception):
    pass

class NonPositivePrice(Exception):
    pass

class InvalidSide(Exception):
    pass

class UndefinedOrderType(Exception):
    pass

class UndefinedOrderSide(Exception):
    pass

class NewQuantityNotSmaller(Exception):
    pass

class UndefinedTraderAction(Exception):
    pass

class UndefinedResponse(Exception):
    pass

Now we introduce necessary classes to define incoming orders:

In [3]:
class Order(ABC):
    def __init__(self, id, symbol, quantity, side, time):
        self.id = id
        self.symbol = symbol
        if quantity > 0:
            self.quantity = quantity
        else:
            raise NonPositiveQuantity("Quantity Must Be Positive!")
        if side in [OrderSide.BUY, OrderSide.SELL]:
            self.side = side
        else:
            raise InvalidSide("Side Must Be Either \"Buy\" or \"OrderSide.SELL\"!")
        self.time = time

class LimitOrder(Order):
    def __init__(self, id, symbol, quantity, price, side, time):
        super().__init__(id, symbol, quantity, side, time)
        if price > 0:
            self.price = price
        else:
            raise NonPositivePrice("Price Must Be Positive!")
        self.type = OrderType.LIMIT

class MarketOrder(Order):
    def __init__(self, id, symbol, quantity, side, time):
        super().__init__(id, symbol, quantity, side, time)
        self.type = OrderType.MARKET

class IOCOrder(Order):
    def __init__(self, id, symbol, quantity, price, side, time):
        super().__init__(id, symbol, quantity, side, time)
        if price > 0:
            self.price = price
        else:
            raise NonPositivePrice("Price Must Be Positive!")
        self.type = OrderType.IOC

class FilledOrder(Order):
    def __init__(self, id, symbol, quantity, price, side, time, limit = False):
        super().__init__(id, symbol, quantity, side, time)
        self.price = price
        self.limit = limit          

Now we introduce the matching engine, which sorts the orders into respective books as elements of a list.

In [21]:
class MatchingEngine():
    def __init__(self):
        self.bid_book = []
        self.ask_book = []

    def handle_order(self, order):
        if order.type==OrderType.IOC:
            MatchingEngine().handle_ioc_order(order)
        elif order.type== OrderType.LIMIT:
            MatchingEngine().handle_limit_order(order)
        elif order.type==OrderType.MARKET:
            MatchingEngine().handle_market_order(order)
        else:
            raise UndefinedOrderType("Undefined Order Type!")

    def handle_limit_order(self, order):  
        filled_orders = []
        if order.side==OrderSide.BUY:
            s=any(num.price<=order.price for num in self.ask_book)
            if s==False:
                self.bid_book.append(order)
                for num in self.bid_book:
                    self.bid_book.sort(key=lambda num:num.price, reverse = True)
            if self.ask_book!=[]:
                if self.ask_book[0].price<=order.price:
                    if self.ask_book[0].quantity<=order.quantity:
                        self.ask_book[0].quantity=self.ask_book[0].quantity-order.quantity
                        order.quantity=0-self.ask_book[0].quantity
                        filled_orders.append(self.ask_book[0])
                        self.ask_book[0].quantity=0
                        self.ask_book.pop(0)
                        filled_orders.append(order)
                    if self.ask_book[0].quantity>order.quantity:
                        self.ask_book[0].quantity=self.ask_book[0].quantity-order.quantity
                        filled_orders.append(self.ask_book[0])
                        filled_orders.append(order)
        elif order.side==OrderSide.SELL:
            s=any(num.price>=order.price for num in self.bid_book)
            if s==False:
                self.ask_book.append(order)
                for num in self.ask_book:
                    self.ask_book.sort(key=lambda num:num.price)
            if self.bid_book!=[]:
                if self.bid_book[0].price>=order.price:
                    if self.bid_book[0].quantity<=order.quantity:
                        self.bid_book[0].quantity=self.bid_book[0].quantity-order.quantity
                        order.quantity=0-self.bid_book[0].quantity
                        filled_orders.append(self.bid_book[0])
                        self.bid_book[0].quantity=0
                        self.bid_book.pop(0)
                        filled_orders.append(order)
                    if self.bid_book[0].quantity>order.quantity:
                        self.bid_book[0].quantity=self.bid_book[0].quantity-order.quantity
                        filled_orders.append(self.bid_book[0])
                        filled_orders.append(order)
        else:
            raise UndefinedOrderSide("Undefined Order Side!")
        return filled_orders       
        
    def insert_limit_order(self, order):
        assert order.type == OrderType.LIMIT
        if order.side==OrderSide.BUY:
            self.bid_book.append(order)
            for num in self.bid_book:
                self.bid_book.sort(key=lambda num:num.price, reverse = True)
        elif order.side==OrderSide.SELL:
            self.ask_book.append(order)
            for num in self.ask_book:
                self.ask_book.sort(key=lambda num:num.price)
        else:
            raise UndefinedOrderSide("Undefined Order Side!")
            
    def handle_market_order(self, order):
        filled_orders = []
        if order.side==OrderSide.BUY:
            if self.ask_book!=[]:
                if self.ask_book[0].quantity<=order.quantity:
                    self.ask_book[0].quantity=self.ask_book[0].quantity-order.quantity
                    order.quantity=0-self.ask_book[0].quantity
                    filled_orders.append(self.ask_book[0])
                    self.ask_book[0].quantity=0
                    self.ask_book.pop(0)
                    filled_orders.append(order)
                if self.ask_book[0].quantity>order.quantity:
                    self.ask_book[0].quantity=self.ask_book[0].quantity-order.quantity
                    filled_orders.append(self.ask_book[0])
                    filled_orders.append(order)
        if order.side==OrderSide.SELL:
             if self.bid_book!=[]:
                if self.bid_book[0].quantity<=order.quantity:
                    self.bid_book[0].quantity=self.bid_book[0].quantity-order.quantity
                    order.quantity=0-self.bid_book[0].quantity
                    filled_orders.append(self.bid_book[0])
                    self.bid_book[0].quantity=0
                    self.bid_book.pop(0)
                    filled_orders.append(order)
                if self.bid_book[0].quantity>order.quantity:
                    self.bid_book[0].quantity=self.bid_book[0].quantity-order.quantity
                    filled_orders.append(self.bid_book[0])
                    filled_orders.append(order)
        else:
            raise UndefinedOrderSide("Undefined Order Side!")      
        return filled_orders
        
    def handle_ioc_order(self, order):
        filled_orders = []
        if order.side==OrderSide.BUY:
            if self.ask_book!=[]:
                if self.ask_book[0].price<=order.price:
                    if self.ask_book[0].quantity<=order.quantity:
                        self.ask_book[0].quantity=self.ask_book[0].quantity-order.quantity
                        order.quantity=0-self.ask_book[0].quantity
                        filled_orders.append(self.ask_book[0])
                        self.ask_book[0].quantity=0
                        self.ask_book.pop(0)
                        filled_orders.append(order)
                    if self.ask_book[0].quantity>order.quantity:
                        self.ask_book[0].quantity=self.ask_book[0].quantity-order.quantity
                        filled_orders.append(self.ask_book[0])
                        filled_orders.append(order)
        if order.side==OrderSide.SELL:
            if self.bid_book!=[]:
                if self.bid_book[0].price>=order.price:
                    if self.bid_book[0].quantity<=order.quantity:
                        self.bid_book[0].quantity=self.bid_book[0].quantity-order.quantity
                        order.quantity=0-self.bid_book[0].quantity
                        filled_orders.append(self.bid_book[0])
                        self.bid_book[0].quantity=0
                        self.bid_book.pop(0)
                        filled_orders.append(order)
                    if self.bid_book[0].quantity>order.quantity:
                        self.bid_book[0].quantity=self.bid_book[0].quantity-order.quantity
                        filled_orders.append(self.bid_book[0])
                        filled_orders.append(order)
        else:
            raise UndefinedOrderSide("Undefined Order Side!")
        return filled_orders
    
    def amend_quantity(self, id, quantity):
        for bid in self.bid_book:
            if bid.id==id:
                if bid.quantity<quantity:
                    raise NewQuantityNotSmaller("Amendment Must Reduce Quantity!")
                else:
                    bid.quantity=quantity   
        for ask in self.ask_book:
            if ask.id==id:
                if bid.quantity<quantity:
                    raise NewQuantityNotSmaller("Amendment Must Reduce Quantity!")
                else:
                    bid.quantity=quantity
    
    def cancel_order(self, id):
        for bid in self.bid_book:
            if bid.id==id:
                index=self.bid_book.index(bid)
                del self.bid_book[index]
        for ask in self.ask_book:
            if ask.id==id:
                index=self.ask_book.index(ask)
                del self.ask_book[index]

Testing the matching engine code above using the unittest package and assertEqual():

In [17]:
import unittest

class TestOrderBook(unittest.TestCase):

    def test_handle_limit_order(self):
        matching_engine = MatchingEngine()
        order = LimitOrder(1, "S", 10, 10, OrderSide.BUY, time.time())
        order_1 = LimitOrder(2, "S", 5, 10, OrderSide.BUY, time.time())
        order_2 = LimitOrder(3, "S", 10, 15, OrderSide.BUY, time.time())
        matching_engine.insert_limit_order(order)
        matching_engine.handle_limit_order(order_1)
        matching_engine.handle_limit_order(order_2)

        self.assertEqual(matching_engine.bid_book[0].price, 15)
        self.assertEqual(matching_engine.bid_book[1].quantity, 10)

        order_sell = LimitOrder(4, "S", 14, 8, OrderSide.SELL, time.time())
        filled_orders = matching_engine.handle_limit_order(order_sell)
        
        self.assertEqual(matching_engine.bid_book[0].price, 10)
        self.assertEqual(matching_engine.bid_book[0].quantity, 6)
        self.assertEqual(filled_orders[0].id, 3)
        self.assertEqual(filled_orders[0].price, 15)
        self.assertEqual(filled_orders[2].id, 1)
        self.assertEqual(filled_orders[2].price, 10)
        
        
    def test_handle_market_order(self):
        matching_engine = MatchingEngine()
        order_1 = LimitOrder(1, "S", 6, 10, OrderSide.BUY, time.time())
        order_2 = LimitOrder(2, "S", 5, 10, OrderSide.BUY, time.time())
        matching_engine.handle_limit_order(order_1)
        matching_engine.handle_limit_order(order_2)
        order = MarketOrder(5, "S", 5, OrderSide.SELL, time.time())
        filled_orders = matching_engine.handle_market_order(order)
        self.assertEqual(matching_engine.bid_book[0].quantity, 1)
        self.assertEqual(filled_orders[0].price, 10)
    
    def test_handle_ioc_order(self):
        matching_engine = MatchingEngine()
        order_1 = LimitOrder(1, "S", 1, 10, OrderSide.BUY, time.time())
        order_2 = LimitOrder(2, "S", 5, 10, OrderSide.BUY, time.time())
        matching_engine.handle_limit_order(order_1)
        matching_engine.handle_limit_order(order_2)
        order = IOCOrder(6, "S", 5, 12, OrderSide.SELL, time.time())
        filled_orders = matching_engine.handle_ioc_order(order)
        self.assertEqual(matching_engine.bid_book[0].price, 10)
        self.assertEqual(matching_engine.bid_book[0].quantity, 1)
        self.assertEqual(len(filled_orders), 0)

In [19]:
TestOrderBook().test_handle_ioc_order()

In [18]:
TestOrderBook().test_handle_limit_order()

In [20]:
TestOrderBook().test_handle_market_order()